In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
from pathlib import Path
from google.colab import drive
import sys

content_path = Path('/').absolute() / 'content'
drive_path = content_path / 'drive'
drive.flush_and_unmount()
drive.mount(str(drive_path))

sys.path.append(str(drive_path / "My Drive" / "OCR" / "utils"))

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from customseg import CustomerStandardizer, CustomerSummary, Periodizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

%matplotlib inline
sns.set_style('white')

In [9]:
df = pd.read_json(drive_path / "My Drive" / "OCR" / "4_Olist" / "orders.json")

In [10]:
prdz = Periodizer(months=3)

df = prdz.fit_transform(df)

In [12]:
df_agg_period = df.groupby("period_id")
periods = dict()
csum = CustomerSummary()
cstd = CustomerStandardizer()

for pid in range(1, df["period_id"].max() + 1):
    periods[pid] = (
        cstd.fit_transform(
            csum.fit_transform(
                df_agg_period.get_group(pid).drop("period_id", axis=1))))